In [1]:
#pplフィルターをinstにかけてみる
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import json
import glob
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()




/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from tqdm import tqdm
model_path="../../llm/models/hf/2jaA"
model_path="../../llm/models/hf/1code"
model_name=model_path.split("/")[-1]

out_path=f"data/gousei_ppl_{model_name}.jsonl"

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:  14%|█▍        | 2/14 [00:02<00:14,  1.19s/it]

In [ ]:
input_jsonl_path="data/gousei.jsonl"
records=[]
with open(input_jsonl_path,"r") as f:
    for line in f:
        records.append(json.loads(line))
len(records)

In [ ]:

with open(out_path,"w") as f:
    f.write("")

cnt=0
with open (out_path,"a") as f:
    for record in tqdm(records):
        cnt+=1
        if cnt>20:
            break
        q=record["text"]
        ppl=perplexity(model,tokenizer,q)
        record["ppl"]=ppl
        #if ppl>30:
        f.write(json.dumps(record,ensure_ascii=False)+"\n")

In [ ]:
import pandas as pd
df=pd.read_json(out_path,lines=True)
df